In [6]:
import os
import pickle
import csv
from pathlib import Path

In [7]:
# Get the path of the file containing the labels
labels_path = './labels.csv'
# Get the path of the file containing the flags of the crawler
flags_path ='./results/crawler_labels.pkl'


In [26]:
def clean_flags(flags):
    for key, value in flags.items():
        flags[key] = value.lower()
    return flags

# Flag is the inferred value, label is the true value
def score_fn(flag, label):
    if label == 'true':
        if flag == 'true':
            return 20
        if flag == 'false':
            return -20
        if flag == 'review':
            return 10
        
    if label == 'false':
        if flag == 'true':
            return -2
        if flag == 'false':
            return 2
        if flag == 'review':
            return -1
        
    raise ValueError(f'Unexpected flags given. Received flag={flag}, label={label}')
        

In [27]:
labels = {}
with open(labels_path, 'r') as handle:
    labels_handle = csv.DictReader(handle, delimiter=',')
    for row in labels_handle:
        labels[row['filename']] = row['cid'].lower()


In [28]:
with open(flags_path, 'rb') as handle:
    flags = pickle.load(handle)

In [29]:
flags = clean_flags(flags)
labels = clean_flags(labels)

In [35]:
n_true = list(labels.values()).count('true')
n_false = list(labels.values()).count('false')

max_score = 20 * n_true + 2 * n_false
min_score = -20 * n_true + -2 * n_false

print(f'Max score: {min_score}')
print(f'Max score: {max_score}')

Max score: -4168
Max score: 4168


In [39]:
score = 0

for filename, label in labels.items():
    # If a file is not labeled or has invalid flags, assume that label "false" was given
    if filename not in flags.keys() or flags[filename] not in ['true', 'false', 'review']:
        score += score_fn('false', label)
    else:
        score += score_fn(flags[filename], label)

normalized_score = (score - min_score) / (max_score - min_score)
print(f'Final score: {score}')
print(f'Final normalized score: {normalized_score}')

Final score: -3000
Final normalized score: 0.1401151631477927
